https://github.com/Teddy-XiongGZ/MedRAG 

* Full corpus of all 4 (50M chunks) - `MedCorp`. BIG DB (over 60GB, even without embedding)
* Lots of space taken by embedding (1gb even for small textpearls)
    *  ` "RRF-2"` - bm25 then dense embed

GPT4 is still better than everything else ;
* https://teddy-xionggz.github.io/benchmark-medical-rag/

MIRAGE Benchmark:
* https://teddy-xionggz.github.io/MIRAGE/

Another relevant *BioRAG* paper (also Medrag components); 
* https://github.com/dmis-lab/self-biorag
* https://www.ncbi.nlm.nih.gov/pmc/articles/PMC11211826/


Possibly better prompt tips:
* https://github.com/microsoft/Phi-3CookBook/blob/main/code/09.UpdateSamples/Aug/phi3_moe_demo.ipynb
* gguf - https://github.com/EricLBuehler/mistral.rs/blob/master/examples/python/gguf.py

In [1]:
# ! conda activate Medrag
# !conda install -c conda-forge openjdk=21 maven nmslib -y
# !conda install -c pytorch faiss-cpu pytorch -y # failed

# ! pip install pyserini  accelerate pyserini datasets  Git-lfs
## for textbooks and gitlfs (wsl): 
#! sudo apt-get install git-lfs
#! git lfs install
# # try? 
# !pip install packaging ninja; MAX_JOBS=4 pip install flash-attn --no-build-isolation 
# !pip install peft bitsandbytes # opt

* answer parsing
* * https://github.com/Teddy-XiongGZ/MIRAGE/blob/main/src/evaluate.py
```
answer_list = ["A", "B", "C", "D"]
            answers.append(locate_fun(it.split('"answer_choice": "')[-1].strip()))
        # answers.append(locate_fun(it.split('"answer_choice": "')[-1].strip()))
        answers = [ans for ans in answers if ans != "NA"]
        if len(answers) == 0:
            pred.append(-1)
            continue
        ans = statistics.mode(answers)
        if ans in answer_list:
            pred.append(answer_list.index(ans))
        else:
            pred.append(-1)
```

Re temp, top_p (if that's being used???)

top_p=1.0

```
Temperature: This parameter determines the creativity and diversity of the text generated by the GPT model. A higher temperature value (e.g., 1.5) leads to more diverse and creative text, while a lower value (e.g., 0.5) results in more focused and deterministic text. 
Top-p Sampling: This parameter maintains a balance between diversity and high-probability words by selecting tokens from the top-p most probable tokens whose collective probability mass is greater than or equal to a threshold p. It helps ensure that the output is both diverse and relevant to the given context. 

Temperature:

If the generated text is too random and lacks coherence, consider lowering the temperature value.
If the generated text is too focused and repetitive, consider increasing the temperature value. 

Top-p Sampling:

If the generated text is too narrow in scope and lacks diversity, consider increasing the probability threshold (p).
If the generated text is too diverse and includes irrelevant words, consider decreasing the probability threshold (p).
```

Could change to : 
```
model.generation_config.temperature=None
model.generation_config.top_p=None
```
or top 1.0 ? 

Some other models officially supported:

```
meta-llama/Meta-Llama-3-70B-Instruct
meta-llama/Llama-2-70b-chat-hf   
mistralai/Mixtral-8x7B-Instruct-v0.1    - 94 GB
epfl-llm/meditron-70b
axiong/PMC_LLaMA_13B
```


Maybe try (assuming they wokr):
* fakezeta/Llama3-OpenBioLLM-8B-ov-int8 - fails to load
    * aaditya/Llama3-OpenBioLLM-8B -  unsupported template
* fakezeta/Meta-Llama-3-70B-Instruct-ov-int4

* https://huggingface.co/mistralai/Mistral-Nemo-Instruct-2407
    *  ? (Mistral nemo, 12B)
 
* medLLM LB:
    * https://huggingface.co/spaces/openlifescienceai/open_medical_llm_leaderboard 

In [2]:
from src.medrag import MedRAG
import pandas as pd
from tqdm import tqdm
import re
from sklearn.metrics import classification_report, roc_auc_score

import json

/home/ddofer/anaconda3/envs/Medrag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### todo: rewrite Q as "is this known already" (instead of "is it unknonw/novel) 

* Issue - models is focuisng on end?
* Maybe remove example of `(For example: Smoking causing lung cancer disease is established (A). A feature of increased smoking preventing gallbladder or cholelithiasis disease would be novel (B))` as model loses focus on question (ands retrieval gets the cancer); TODO: could rewrite with our disease! 

In [3]:
# FAST = True
FAST = False

SAVE_OUTPUT = True#False

# NOVEL_OUTPUT_FILENAME = "novel_res_8b_knownPrompt_64.csv"
# PLAUSIBLE_OUTPUT_FILENAME = "plausible_res_8b_knownPrompt_v1.csv"

NOVEL_OUTPUT_FILENAME = "novel_res_8b_4bit.csv"
PLAUSIBLE_OUTPUT_FILENAME = "plausible_res_4b_8bit.csv"

# NOVEL_OUTPUT_FILENAME = "novel_res_3b.csv"
# PLAUSIBLE_OUTPUT_FILENAME = "plausible_res_3b.csv"

In [4]:
LL_NAME = "meta-llama/Meta-Llama-3.1-8B-Instruct" # works, less stable than 3.0 model?
# LL_NAME ="meta-llama/Llama-3.2-3B-Instruct"
# LL_NAME ="mistralai/Ministral-8B-Instruct-2410"
# LL_NAME = "Zyphra/Zamba2-7B-instruct" # unsupported chat 

 # LL_NAME =  = "meta-llama/Meta-Llama-3-70B-Instruct" 
# LL_NAME = "aaditya/Llama3-OpenBioLLM-8B" # unsupported template
# LL_NAME ="meta-llama/Llama-3.2-3B-Instruct-QLORA_INT4_EO8"

if FAST:
    # LL_NAME ="meta-llama/Llama-3.2-1B-Instruct"
    LL_NAME ="meta-llama/Llama-3.2-3B-Instruct"
    SAVE_OUTPUT = False

In [5]:
# # %%time
# %time cot = MedRAG(llm_name=LL_NAME, rag=False)
# %time answer, _, _ = cot.answer(question=question, options=options)
# print(answer)

In [6]:
# %%time
# medrag = MedRAG(llm_name=LL_NAME, rag=True, 
#                 retriever_name= "MedCPT",
#                 # retriever_name="MedCPT",
#                 corpus_name= "MedText", #"StatPearls", 
#                  # corpus_name="MedCorp", # 3.5 hours/ Failure-crash
#                 corpus_cache=False#True,
#                 ,HNSW=True
#                )
# answer, snippets, scores = medrag.answer(question=question, options=options, k=16)
# answer

----------------

In [7]:
df = pd.read_csv("sample_question_prompts_chol.csv")#.tail(8)
##
## mayeb strengthen "this is a candidate/given feature
df["question"] = df["question"].str.replace(r", SHAP importance 0\.\d{1,8}","",regex=True).str.replace("is this feature","is this feature - disease linkage")


# invert - need to flip options
##  in the existing literature or scientific understanding
# #  \n(For example: Smoking causing lung disease is known (A. True)
# (and directionality)
df["question"] = df["question"].str.replace(", is this feature - disease linkage Novel and interesting?",". Is this feature to disease connection (or directional effect) already known or established?")

#. A feature of increased smoking preventing gallbladder or cholelithiasis disease would be novel (B. False))
df2 = df.dropna(subset=["Plausible/Makes Sense?"]).reset_index(drop=True).copy()
df2["Plausible/Makes Sense?"] = df2["Plausible/Makes Sense?"].astype(bool)

## interesting/novel subset (harder):
df = df.dropna(subset=["Interesting?"]).reset_index(drop=True).copy()
df["Interesting?"] = df["Interesting?"].astype(bool)

if FAST:
    df = df.head()
    df2 = df2.tail()
print(df["question"][0])
df

Given the feature 'Long-standing illness, disability or infirmity Yes'; (p-value: 0.0, correlation: 0.049), as a predictive feature, or risk/protective factor for target disease: 'Cholelithiasis OR Gallstones OR Gallbladder disease OR cholecystitis OR Cholangitis'. Is this feature to disease connection (or directional effect) already known or established?


,question,Interesting?,Plausible/Makes Sense?
0,"Given the feature 'Long-standing illness, disa...",True,NaN
1,Given the feature 'Alanine aminotransferase'; ...,False,NaN
2,Given the feature 'Mouth/teeth dental problems...,True,NaN
3,Given the feature 'Overall health rating Fair'...,False,NaN
4,Given the feature 'Standard PRS for alzheimer'...,True,NaN
...,...,...,...
79,Given the feature '3-Hydroxybutyrate'; (p-valu...,False,NaN
80,Given the feature 'Hallux valgus (Bunion)'; (p...,False,NaN
81,Given the feature 'Cholesterol in Chylomicrons...,True,NaN
82,Given the feature 'Job code at visit clerks'; ...,False,NaN


In [8]:
display(df2)
# print(df2["question"][0])

,question,Interesting?,Plausible/Makes Sense?
0,"Given the feature 'Long-standing illness, disa...",NaN,True
1,Given the feature 'Alanine aminotransferase'; ...,NaN,True
2,Given the feature 'Mouth/teeth dental problems...,NaN,True
3,Given the feature 'Overall health rating Fair'...,NaN,False
4,Given the feature 'Standard PRS for alzheimer'...,NaN,True
...,...,...,...
79,Given the feature '3-Hydroxybutyrate'; (p-valu...,NaN,False
80,Given the feature 'Hallux valgus (Bunion)'; (p...,NaN,False
81,Given the feature 'Cholesterol in Chylomicrons...,NaN,False
82,Given the feature 'Job code at visit clerks'; ...,NaN,False


Bad example of model doing opposite and bad COT leading to that:

```
    is this feature Novel and interesting? Here are the potential choices: A. True. Novel B. False. Already known or irrelevant or nonsense or confounded Please think step-by-step and generate your output in json:<|eot_id|><|start_header_id|>assistant<|end_header_id|> Here is the output in JSON format: { "step_by_step_thinking": "The feature 'Mouth/teeth dental problems Dentures' is not mentioned in any of the provided documents as a risk factor or protective factor for cholelithiasis, gallstones, gallbladder disease, cholecystitis, or cholangitis. Therefore, it is not a novel or interesting feature for predicting these diseases. The documents do mention various risk factors such as female gender, obesity, rapid weight loss, pregnancy, advanced age, Hispanic or Pima Indian ethnicity, and sickle cell disease, but dental problems are not mentioned. Therefore, the correct answer is B. False. Already known or irrelevant or nonsense or confounded.", "answer_choice": B }

```

Another odd reasoning:

```
 B. False. Already known risk factor or cause }
": "After reviewing the provided documents, I did not find any mention of 'Mouth/teeth dental problems Dentures' as a risk factor or cause for cholelithiasis, gallstones, gallbladder disease, cholecystitis, or cholangitis. The documents primarily focus on the pathophysiology, diagnosis, and treatment of these conditions, as well as risk factors such as age, sex, obesity, and other medical conditions. There is no mention of dental problems or dentures as a potential risk factor or cause for these conditions. Therefore, I conclude that this feature is not novel or previously unstudied.", "
```


```
 B. False. Established, known risk factor or cause, for this disease
": "I reviewed the provided documents and did not find any mention of 'Standing height' as a predictive feature or risk/protective factor for cholelithiasis, gallstones, gallbladder disease, cholecystitis, or cholangitis. Therefore, I conclude that this feature is not a known risk factor or cause for these diseases.", "
```

In [9]:
# medrag = MedRAG(llm_name=LL_NAME, rag=True,
#                 retriever_name="BM25" # "BM25" #"RRF-2"#"MedCPT" - RRF2 woudl take ~ 11 hours for pubmed
# , corpus_name= "MedCorp",HNSW=True)

In [10]:
%%time
medrag = MedRAG(llm_name=LL_NAME, rag=True,
                retriever_name="BM25" # "BM25" #"RRF-2"#"MedCPT" - RRF2 woudl take ~ 11 hours for pubmed
, corpus_name= "MedCorp"
                ,HNSW=True
               )

plausible_pred = []
plausible_cot = []
plausible_answers_full = []
plausible_answers_json = []
for index, row in tqdm(df2.iterrows(), total=df2.shape[0], desc="Processing rows"):
    question = row['question']
    options = {
    "A": "True. Plausible, makes sense", # Already known or irrelevant or nonsense or confounded
    "B": "False. No plausible link or explanation"} # mechanismin relation to this disease # ,interesting

    answer, snippets, scores = medrag.answer(question=question, options=options, k=4 if FAST else 32) #32
    ## m aybe>? (handle missing/closing "}" hack
    # Ensure the answer ends with a single closing curly brace
    if not answer.endswith("}"):
        answer += "}"
    
    # Use regex to replace two or more consecutive closing curly braces with just one
    answer = re.sub(r'\}+', '}', answer)
    
    try:
        # json_ans = json.loads(answer)
        json_ans = json.loads(re.search('{.+}', answer, re.IGNORECASE).group(0))
        
    except:
        json_ans = json.dumps({'step_by_step_thinking':None,'answer_choice':None})
        print("Failed parse")
        print(answer)

    try:
        cot = json_ans['step_by_step_thinking']
        pred = json_ans['answer_choice']
        print(cot)
        # print(pred)
    except:
        pred =answer.split('answer_choice')[-1] 
        cot = answer.split("step_by_step_thinking")[-1].split("answer_choice")[0]

    plausible_pred.append(pred)
    plausible_cot.append(cot)
    plausible_answers_full.append(answer)
    plausible_answers_json.append(json_ans)
    # plausible_answers_snippets.append(snippets)

Processing rows:   0%|                                                                                        | 0/84 [00:00<?, ?it/s]/home/ddofer/anaconda3/envs/Medrag/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/ddofer/anaconda3/envs/Medrag/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/ddofer/anaconda3/envs/Medrag/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.w

To determine if the feature 'Long-standing illness, disability or infirmity Yes' is a plausible risk factor for cholelithiasis or gallbladder disease, we need to consider the relationship between chronic illness and the development of gallstones. 1. Chronic illness, disability, or infirmity can lead to changes in the body's metabolism, including changes in bile composition and gallbladder function. 2. These changes can increase the risk of gallstone formation, as the gallbladder may become less effective at emptying bile and more prone to supersaturation with cholesterol or other substances. 3. Additionally, chronic illness may lead to a decrease in physical activity, which can also contribute to the development of gallstones. 4. Furthermore, some chronic illnesses, such as diabetes, may be associated with an increased risk of gallstones due to factors such as insulin resistance and altered bile composition. Considering these points, it is plausible that long-standing illness, disabili

Processing rows:   2%|█▊                                                                           | 2/84 [03:46<2:26:22, 107.11s/it]

To determine if the feature 'Alanine aminotransferase' is a plausible predictive feature or risk/protective factor for the target diseases 'Cholelithiasis', 'Gallstones', 'Gallbladder disease', 'Cholecystitis', and 'Cholangitis', we need to consider the relationship between Alanine aminotransferase (ALT) levels and the development or progression of these conditions. ALT is an enzyme found in the liver, and elevated levels are often associated with liver damage or disease. In the context of gallbladder disease, elevated ALT levels could indicate liver involvement or damage, which may be secondary to the gallbladder disease. For example, in the case of cholangitis, inflammation of the bile ducts, elevated ALT levels could reflect liver damage caused by the infection. Similarly, in the case of cholelithiasis, elevated ALT levels could indicate liver damage caused by the gallstones. Therefore, a positive correlation between ALT levels and the target diseases is plausible, as elevated ALT l

Processing rows:   4%|██▊                                                                           | 3/84 [04:45<1:55:21, 85.45s/it]

...

To determine if the feature 'Cholesterol in Chylomicrons and Extremely Large VLDL' is a plausible risk or protective factor for cholelithiasis, gallstones, gallbladder disease, cholecystitis, or cholangitis, we need to consider the relationship between this feature and the formation of gallstones. 1. Chylomicrons are lipoproteins that transport dietary triglycerides from the intestines to the liver. They are not directly related to the formation of cholesterol gallstones. 2. VLDL (Very Low-Density Lipoprotein) is a type of lipoprotein that transports endogenous triglycerides, cholesterol, and other lipids from the liver to peripheral tissues. Extremely large VLDL particles are associated with an increased risk of cardiovascular disease, but their direct relationship to gallstone formation is unclear. 3. The formation of cholesterol gallstones is a complex process involving an imbalance between the secretion of cholesterol and bile salts by the liver, impaired gallbladder emptying, and 

Processing rows:  99%|████████████████████████████████████████████████████████████████████████████ | 83/84 [1:24:06<00:55, 55.99s/it]

To determine if the feature 'Job code at visit clerks' is a plausible risk or protective factor for gallbladder disease, we need to consider the potential relationship between job code and gallbladder disease. The provided p-value of 0.08 and correlation of -0.003 suggest a weak association between the two variables. However, the correlation is negative, indicating that as the job code at visit clerks increases, the risk of gallbladder disease decreases. This is counterintuitive, as one would expect a positive correlation if job code had a direct impact on gallbladder disease. Furthermore, there is no clear mechanism by which job code at visit clerks would affect gallbladder disease. Job code is related to administrative tasks and does not have a direct link to gallbladder function, bile composition, or other factors that contribute to gallstone formation. Therefore, it is unlikely that job code at visit clerks is a risk or protective factor for gallbladder disease.


Processing rows: 100%|█████████████████████████████████████████████████████████████████████████████| 84/84 [1:24:56<00:00, 60.67s/it]

The question is asking whether the feature 'Other diseases of respiratory system, NEC' is a plausible risk or protective factor for cholelithiasis, gallstones, gallbladder disease, cholecystitis, or cholangitis. To determine this, we need to consider if there is a known mechanism by which respiratory diseases could affect the development of gallbladder disease. The provided documents do not mention any direct link between respiratory diseases and gallbladder disease. However, we can consider the potential indirect effects of respiratory diseases on the body that could influence gallbladder function or disease development. For example, chronic respiratory diseases such as chronic obstructive pulmonary disease (COPD) may lead to malnutrition, which can affect bile composition and increase the risk of gallstone formation. Additionally, some respiratory diseases may be associated with systemic inflammation, which could potentially affect the gallbladder. However, the provided p-value and c

In [11]:
# json.loads(re.search('{.+}', answer, re.IGNORECASE).group(0))

In [12]:
df_plausible_pred_small = pd.DataFrame({"question":df2["question"],"Plausible/Makes Sense?":df2["Plausible/Makes Sense?"] ,
                                    "pred_novel":plausible_pred,"pred_plausible_cot":plausible_cot,"plausible_full":plausible_answers_full,
                                   })

df_plausible_pred_small["pred_plausible"] = df_plausible_pred_small["pred_novel"].str.replace("[^A-B]"," ",case=False,regex=True).str.strip()
df_plausible_pred_small["pred_plausible"] = df_plausible_pred_small["pred_plausible"].str.startswith("A") # NOTE! Fails to account for bad formats!

display(df_plausible_pred_small)
#ALT? 
# df_plausible_pred_small["pred_plausible"] = df_plausible_pred_small["pred_novel"].str.contains("B\.",regex=True)

print(classification_report(df_plausible_pred_small["Plausible/Makes Sense?"],df_plausible_pred_small["pred_plausible"]))
print(df_plausible_pred_small[["Plausible/Makes Sense?","pred_plausible"]].corr())
if not FAST: 
    
    print("rAUC",round(100*roc_auc_score(df_plausible_pred_small["Plausible/Makes Sense?"],df_plausible_pred_small["pred_plausible"]),3))
if SAVE_OUTPUT:
    df_plausible_pred_small.to_csv(PLAUSIBLE_OUTPUT_FILENAME,index=False)


,question,Plausible/Makes Sense?,pred_novel,pred_plausible_cot,plausible_full,pred_plausible
0,"Given the feature 'Long-standing illness, disa...",True,"A. True. Plausible, makes sense",To determine if the feature 'Long-standing ill...,"{ ""step_by_step_thinking"": ""To determine if th...",True
1,Given the feature 'Alanine aminotransferase'; ...,True,"A. True. Plausible, makes sense",To determine if the feature 'Alanine aminotran...,"{ ""step_by_step_thinking"": ""To determine if th...",True
2,Given the feature 'Mouth/teeth dental problems...,True,A,To determine if the feature 'Mouth/teeth denta...,"{ ""step_by_step_thinking"": ""To determine if th...",True
3,Given the feature 'Overall health rating Fair'...,False,B. False. No plausible link or explanation,To determine if the feature 'Overall health ra...,"{ ""step_by_step_thinking"": ""To determine if th...",False
4,Given the feature 'Standard PRS for alzheimer'...,True,B. False. No plausible link or explanation,To determine if the feature 'Standard PRS for ...,"{ ""step_by_step_thinking"": ""To determine if th...",False
...,...,...,...,...,...,...
79,Given the feature '3-Hydroxybutyrate'; (p-valu...,False,B. False. No plausible link or explanation,To determine if the feature '3-Hydroxybutyrate...,"{ ""step_by_step_thinking"": ""To determine if th...",False
80,Given the feature 'Hallux valgus (Bunion)'; (p...,False,B. False. No plausible link or explanation,To determine if the feature 'Hallux valgus (Bu...,"{ ""step_by_step_thinking"": ""To determine if th...",False
81,Given the feature 'Cholesterol in Chylomicrons...,False,B. False. No plausible link or explanation,To determine if the feature 'Cholesterol in Ch...,"{ ""step_by_step_thinking"": ""To determine if th...",False
82,Given the feature 'Job code at visit clerks'; ...,False,B. False. No plausible link or explanation,To determine if the feature 'Job code at visit...,"{""step_by_step_thinking"": ""To determine if the...",False


              precision    recall  f1-score   support

       False       0.64      0.78      0.70        45
        True       0.66      0.49      0.56        39

    accuracy                           0.64        84
   macro avg       0.65      0.63      0.63        84
weighted avg       0.65      0.64      0.63        84

                        Plausible/Makes Sense?  pred_plausible
Plausible/Makes Sense?                1.000000        0.277929
pred_plausible                        0.277929        1.000000
rAUC 63.248


In [9]:
%%time
medrag = MedRAG(llm_name=LL_NAME, rag=True,
                retriever_name="BM25" # "BM25" #"RRF-2"#"MedCPT" - RRF2 woudl take ~ 11 hours for pubmed
, corpus_name= "MedCorp",
                HNSW=True
               )

novel_pred = []
novel_cot = []
novel_answers_full = []
# novel_answers_snippets = []
novel_answers_json = []
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
    question = row['question']
    options = {
    # "A": "True. Potentially Novel, previously unstudied or unknown in existing literature", # ,interesting
    # "B": "False. Established, known risk factor or cause"} # Already known or irrelevant or nonsense or confounded
    # "A": "True. Potentially Novel, previously unstudied or unknown in existing literature for this disease", # ,interesting
    # "B": "False. Established, known risk factor or cause, for this disease"} # Already known or irrelevant or nonsense or confounded
## flip:
     "A": "True. Known, or established or risk factor for this disease",#, symptom, ", #/etiology # Already known or irrelevant or nonsense or confounded
    "B": "False. Novel, unknown, controversial or unstudied"#, for this disease"} # in relation to this disease # ,interesting
    }
    # answer, _, _ = cot.answer(question=question, options=options)
    answer, snippets, scores = medrag.answer(question=question, options=options, k=4 if FAST else 33) #32

    # Ensure the answer ends with a single closing curly brace
    if not answer.endswith("}"):
        answer += "}"
    
    # Use regex to replace two or more consecutive closing curly braces with just one
    answer = re.sub(r'\}+', '}', answer)
 
    try:
        json_ans = json.loads(re.search('{.+}', answer, re.IGNORECASE).group(0))
        
    except:
        json_ans = json.dumps({'step_by_step_thinking':None,'answer_choice':None})
        # plausible_answers_json.append(json.dumps({}) )## append empty json
        print("Failed parse")
        print(answer)

    try:
        cot = json_ans['step_by_step_thinking']
        pred = json_ans['answer_choice']
        print(cot)
        # print(pred)
    except:
        pred =answer.split('answer_choice')[-1] 
        cot = answer.split("step_by_step_thinking")[-1].split("answer_choice")[0]

    novel_pred.append(pred)
    novel_cot.append(cot)
    novel_answers_full.append(answer)
    # novel_answers_snippets.append(snippets)
    novel_answers_json.append(json_ans)

Processing rows:   0%|                                                                                        | 0/84 [00:00<?, ?it/s]/home/ddofer/anaconda3/envs/Medrag/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/ddofer/anaconda3/envs/Medrag/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/ddofer/anaconda3/envs/Medrag/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.w

To determine if the feature 'Long-standing illness, disability or infirmity Yes' is a known or established risk factor for the target diseases, we need to analyze the provided documents. The feature is mentioned in Document [30] as a risk factor for symptomatic gallstones among young women. Additionally, Document [18] mentions that gallstones are more common in people with severe motor and intellectual disabilities (SMID), indicating a potential link between long-standing illness or disability and gallstone formation. Furthermore, Document [29] discusses the association between primary sclerosing cholangitis (PSC) and extrahepatic biliary tract disease, which may be related to long-standing illness or disability. However, the specific connection between 'Long-standing illness, disability or infirmity Yes' and the target diseases is not extensively discussed in the provided documents. Therefore, we need to consider the correlation value of 0.049, which suggests a moderate association be

Processing rows:   2%|█▊                                                                           | 2/84 [03:23<2:18:34, 101.39s/it]

To determine if the feature 'Alanine aminotransferase' is a known or established risk factor for the target diseases, we need to analyze the provided documents. The feature 'Alanine aminotransferase' is mentioned in several documents as a lab finding that can be associated with liver disease. Document [15] states that 'Alanine aminotransferase and aspartate transaminase are usually suggestive of liver disease.' This suggests that elevated levels of Alanine aminotransferase can be indicative of liver disease, which is a related condition to the target diseases. Document [16] also mentions that 'Gallstones within the ampulla of Vater can obstruct the exocrine system of the pancreas, which in turn can result in pancreatitis.' Pancreatitis is a complication of gallstone disease, and elevated Alanine aminotransferase levels can be associated with pancreatitis. Therefore, it is reasonable to conclude that Alanine aminotransferase is a known risk factor for the target diseases.


Processing rows:   4%|██▊                                                                           | 3/84 [04:28<1:54:46, 85.02s/it]

...

To determine if the feature 'Cholesterol in Chylomicrons and Extremely Large VLDL' is a known or established risk factor for 'Cholelithiasis OR Gallstones OR Gallbladder disease OR cholecystitis OR Cholangitis', we need to analyze the relationship between the two. The given p-value of 0.0 and correlation of 0.002 suggest a strong association between the feature and the disease. However, the direction of the effect (risk or protective) is not specified. We need to look for studies or evidence that have investigated this specific relationship. Upon reviewing the provided documents, we find that Document [13] mentions that statins inhibit cholesterol synthesis and may decrease the risk of getting gallstones, which suggests that cholesterol levels can affect gallstone formation. However, Document [12] states that high levels of total cholesterol increase the risk of cardiovascular disease, particularly coronary heart disease. Document [13] also mentions that fibrates increase cholesterol c

Processing rows:  99%|████████████████████████████████████████████████████████████████████████████ | 83/84 [1:39:04<01:06, 66.55s/it]

To determine if the feature 'Job code at visit clerks' is a known or established risk factor for the target diseases (Cholelithiasis, Gallstones, Gallbladder disease, Cholecystitis, Cholangitis), we need to consider the available information. The provided documents do not mention 'Job code at visit clerks' as a risk factor for any of these diseases. However, we can look for related factors that might be associated with job codes or work-related activities. For example, some studies suggest that prolonged sitting or sedentary behavior, which might be related to certain job codes, can increase the risk of gallstones. Additionally, stress, which can be related to certain job codes, has been linked to an increased risk of gallstones. However, these are indirect associations and not directly related to 'Job code at visit clerks'. Therefore, based on the available information, we cannot confirm that 'Job code at visit clerks' is a known or established risk factor for the target diseases.


Processing rows: 100%|█████████████████████████████████████████████████████████████████████████████| 84/84 [1:40:00<00:00, 71.44s/it]

To determine if the feature 'Other diseases of respiratory system, NEC' is a known or established risk factor for the target diseases 'Cholelithiasis OR Gallstones OR Gallbladder disease OR cholecystitis OR Cholangitis', we need to analyze the provided documents. The documents do not directly mention a connection between 'Other diseases of respiratory system, NEC' and the target diseases. However, we can look for indirect connections or associations. Document [11] mentions that Crohn's disease, which affects the ileum, may result in an increased risk of gallstones due to a decrease in bile acid resorption in the ileum. This suggests that gastrointestinal diseases can be associated with gallstone formation. Document [29] lists conditions affecting the pancreas, bile salts, and intestinal malabsorption as causes of steatorrhea, which is a symptom of gallbladder disease. Document [31] lists abdominal pain caused by various conditions, including gastrointestinal, vascular, and glandular di

In [10]:
df_novel_pred_small = pd.DataFrame({"question":df["question"],"Interesting?":df["Interesting?"].astype(bool),
                                    "pred_novel":novel_pred,"pred_novel_cot":novel_cot,
                                    "novel_full":novel_answers_full,
                                    "novel_answers_json":novel_answers_json
                                   })

df_novel_pred_small["pred_Interesting"] = df_novel_pred_small["pred_novel"].str.replace("[^A-B]"," ",case=False,regex=True).str.strip()
df_novel_pred_small["pred_Interesting"] = df_novel_pred_small["pred_Interesting"].str.startswith("B") # NOTE! Fails to account for bad formats!
print(df_novel_pred_small["pred_Interesting"].value_counts(normalize=True).round(2))
print(classification_report(df_novel_pred_small["Interesting?"],df_novel_pred_small["pred_Interesting"]))
if not FAST: print(round(100*roc_auc_score(df_novel_pred_small["Interesting?"],df_novel_pred_small["pred_Interesting"]),3))

if SAVE_OUTPUT:
    df_novel_pred_small.to_csv(NOVEL_OUTPUT_FILENAME,index=False)
display(df_novel_pred_small)

pred_Interesting
True     0.88
False    0.12
Name: proportion, dtype: float64
              precision    recall  f1-score   support

       False       0.30      0.08      0.12        38
        True       0.53      0.85      0.65        46

    accuracy                           0.50        84
   macro avg       0.41      0.46      0.39        84
weighted avg       0.42      0.50      0.41        84

46.339


,question,Interesting?,pred_novel,pred_novel_cot,novel_full,novel_answers_json,pred_Interesting
0,"Given the feature 'Long-standing illness, disa...",True,A,To determine if the feature 'Long-standing ill...,"{ ""step_by_step_thinking"": ""To determine if th...",{'step_by_step_thinking': 'To determine if the...,False
1,Given the feature 'Alanine aminotransferase'; ...,False,"A. True. Known, or established or risk factor ...",To determine if the feature 'Alanine aminotran...,"{ ""step_by_step_thinking"": ""To determine if th...",{'step_by_step_thinking': 'To determine if the...,False
2,Given the feature 'Mouth/teeth dental problems...,True,"B. False. Novel, unknown, controversial or uns...",To determine if the feature 'Mouth/teeth denta...,"{ ""step_by_step_thinking"": ""To determine if th...",{'step_by_step_thinking': 'To determine if the...,True
3,Given the feature 'Overall health rating Fair'...,False,"B. False. Novel, unknown, controversial or uns...",To determine if the feature 'Overall health ra...,"{ ""step_by_step_thinking"": ""To determine if th...",{'step_by_step_thinking': 'To determine if the...,True
4,Given the feature 'Standard PRS for alzheimer'...,True,"B. False. Novel, unknown, controversial or uns...",To determine if the feature 'Standard PRS for ...,"{ ""step_by_step_thinking"": ""To determine if th...",{'step_by_step_thinking': 'To determine if the...,True
...,...,...,...,...,...,...,...
79,Given the feature '3-Hydroxybutyrate'; (p-valu...,False,""": ""B. False. Novel, unknown, controversial or...",""": ""To determine if the feature '3-Hydroxybuty...","{ ""step_by_step_thinking"": ""To determine if th...","{""step_by_step_thinking"": null, ""answer_choice...",True
80,Given the feature 'Hallux valgus (Bunion)'; (p...,False,"B. False. Novel, unknown, controversial or uns...",To determine if Hallux valgus (Bunion) is a kn...,"{ ""step_by_step_thinking"": ""To determine if Ha...",{'step_by_step_thinking': 'To determine if Hal...,True
81,Given the feature 'Cholesterol in Chylomicrons...,True,"B. False. Novel, unknown, controversial or uns...",To determine if the feature 'Cholesterol in Ch...,"{ ""step_by_step_thinking"": ""To determine if th...",{'step_by_step_thinking': 'To determine if the...,True
82,Given the feature 'Job code at visit clerks'; ...,False,"B. False. Novel, unknown, controversial or uns...",To determine if the feature 'Job code at visit...,"{ ""step_by_step_thinking"": ""To determine if th...",{'step_by_step_thinking': 'To determine if the...,True


### run with bigger model

MIXTRAL: needs: 
* https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/discussions/143
* `transformers==4.41.2` +

* https://www.kaggle.com/code/ashishkumarak/mixtral-moe-8x7b-instruct-inference-t4-2-gpu

Could try gguf mixtral?
* https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF
* https://stackoverflow.com/questions/77630013/how-to-run-any-gguf-model-using-transformers-or-any-other-library
* `pip install gguf`
```
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF"
filename = "tinyllama-1.1b-chat-v1.0.Q6_K.gguf"

tokenizer = AutoTokenizer.from_pretrained(model_id, gguf_file=filename)
model = AutoModelForCausalLM.from_pretrained(model_id, gguf_file=filename)
```
*  https://github.com/EricLBuehler/mistral.rs/blob/master/examples/python/cookbook.ipynb
    *   big models loading + chat example, inc. ggug

* phi-moe - similar to mixtral , 86GB ; https://huggingface.co/microsoft/Phi-3.5-MoE-instruct
    *  Too big even with 4bit and gpu+cpu, in hf/bnb. (Maybe doable with others)
* `microsoft/Phi-3-medium-128k-instruct` - 14B model
* `microsoft/Phi-3-mini-128k-instruct` - ~4B model, fast

* `microsoft/Phi-3-small-128k-instruct`   - 7B model

    *  PHI-3 cookbook, llm ft + generation examples - https://github.com/microsoft/Phi-3CookBook/blob/main/code/04.Finetuning/Phi_3_Inference_Finetuning.ipynb
https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing
* Unsloth, 4bit large model loading

* Try more indepth RAG (dense +- RRF) - but smaller corpus

In [11]:
# %%time
# # LL_NAME =  "meta-llama/Meta-Llama-3-70B-Instruct" 
# # LL_NAME =  "mistralai/Mixtral-8x7B-Instruct-v0.1" ## too big
# # LL_NAME =  "meta-llama/Llama-3.2-1B" # unsupported - no chat template 
# # LL_NAME ="microsoft/Phi-3-mini-128k-instruct" # works; not evaluated
LL_NAME ="microsoft/Phi-3-medium-128k-instruct"

# # LL_NAME = "BioMistral/BioMistral-7B" # unsupported format/Conversation roles : system/assistant instead of  system/user
# # LL_NAME = "axiong/PMC_LLaMA_13B" ## bad answer formatting

# # LL_NAME = "nvidia/Mistral-NeMo-12B-Instruct" # unsupported
# # LL_NAME = "mistralai/Mistral-Nemo-Instruct-2407"

# # LL_NAME = "microsoft/Phi-3.5-MoE-instruct" ## too big!
# ## GGUF version - may not work? but saner vram reqs;
# ## https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF 
# ## https://www.kaggle.com/code/ashishkumarak/mixtral-moe-8x7b-instruct-inference-t4-2-gpu

# medrag = MedRAG(llm_name=LL_NAME, rag=True,
#                 retriever_name="MedCPT"
# , corpus_name= "MedCorp",HNSW=True
#                )
medrag = MedRAG(llm_name=LL_NAME, rag=True,
                retriever_name="MedCPT"
, corpus_name= "MedText",
                HNSW=True
                # ,corpus_cache=True # uses a lot of ram
               )

big_plausible_pred = []
big_plausible_cot = []
big_plausible_answers_full = []
# big_novel_answers_snippets = []

# for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
#     question = row['question']
#     options = {
#     # "A": "True. Potentially Novel, previously unstudied or unknown in existing literature, in relation to this disease", # ,interesting
#     # "B": "False. Established, known risk factor or cause for this disease"} # Already known or irrelevant or nonsense or confounded
# ## flip:
#     #  "A": "True. Established, known risk factor, symptom cause for this disease", # Already known or irrelevant or nonsense or confounded
#     # "B": "False. Potentially Novel, previously unstudied or unknown in existing literature, in relation to this disease"} # ,interesting
#      "A": "True. Established, or known for this disease", # Already known or irrelevant or nonsense or confounded
#     "B": "False. Novel, unknown, controversial or unstudied in existing literature, for this disease"} # ,interesting

#     answer, snippets, scores = medrag.answer(question=question, options=options, k=36)
#     # if len(row["snippets"]>0: # use alread ysaved snippets:
#     # answer, snippets, scores = medrag.answer(question=question, options=options, 
#     #                                          snippets=big_novel_answers_snippets[index] ## maybe not wokring ok? also it has ids in there.. 
#     #                                         )

#     # answer = answer.split("<|eot_id|><|start_header_id|>assistant<|end_header_id|> Here is the output in JSON format: {")[-1]
#     answer = answer.split("<|eot_id|><|start_header_id|>assistant<|end_header_id|>")[-1]
#     answer = answer.split("Here is the output in JSON format: {")[-1]
#     pred = answer.split('answer_choice":')[-1] # 
#     pred_cot = answer.split("step_by_step_thinking")[-1].split("answer_choice")[0]
#     print(pred)
#     print(pred_cot)
#     big_novel_pred.append(pred)
#     big_novel_cot.append(pred_cot)
#     big_novel_answers_full.append(answer)


for index, row in tqdm(df2.iterrows(), total=df2.shape[0], desc="Processing rows"):
    question = row['question']
    options = {
    "A": "True. Plausible or makes sense", # Already known or irrelevant or nonsense or confounded
    "B": "False. No plausible link, explanation or mechanism"} # in relation to this disease # ,interesting

    answer, snippets, scores = medrag.answer(question=question, options=options, k=4 if FAST else 36) #32
    ## m aybe>? (handle missing/closing "}" hack
    # Ensure the answer ends with a single closing curly brace
    if not answer.endswith("}"):
        answer += "}"
    
    # Use regex to replace two or more consecutive closing curly braces with just one
    answer = re.sub(r'\}+', '}', answer)
    
    ## phi gives different output format - handle it by making us expect it and not regular:
    answer = answer.replace("answer_choice","answer")
    answer = answer.replace("step_by_step_thinking","explanation")

    answer = re.sub(r'{ ', '{', answer)
    answer = re.sub(r' }', '}', answer)

    try:
        # json_ans = json.loads(answer)
        json_ans = json.loads(re.search('{.+}', answer, re.IGNORECASE).group(0))
        
    except:
        json_ans = json.dumps({'explanation':None,'answer':None})
        print("Failed parse")
        print(answer)
    try:
        cot = json_ans['explanation']
        pred = json_ans['answer']
        print(cot)
        # print(pred)
    except:
        pred =answer.split('answer')[-1] 
        cot = answer.split("explanation")[-1].split("answer")[0]

    big_plausible_pred.append(pred)
    big_plausible_cot.append(cot)
    big_plausible_answers_full.append(answer)
    # big_plausible_answers_json.append(json_ans)
    # plausible_answers_snippets.append(snippets)

No sentence-transformers model found with name ncbi/MedCPT-Query-Encoder. Creating a new one with CLS pooling.
No sentence-transformers model found with name ncbi/MedCPT-Query-Encoder. Creating a new one with CLS pooling.


Processing rows:   0%|                                                                                        | 0/84 [00:00<?, ?it/s]/home/ddofer/anaconda3/envs/Medrag/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Processing rows:   1%|▉                                                                            | 1/84 [02:41<3:43:34, 161.62s/it]

Failed parse
 {"answer": "A", "explanation": "Long-standing illness, disability, or infirmity can be associated with a higher risk of developing gallstones or gallbladder disease. This is because chronic conditions can lead to changes in metabolism, diet, and physical activity, which can contribute to the formation of gallstones. Additionally, certain medications used to treat chronic conditions may also increase the risk of gallstones. Therefore, there is a plausible mechanism for the feature to affect the target disease."} {"answer": "A", "explanation": "Long-standing illness, disability, or infirmity can be associated with a higher risk of developing gallstones or gallbladder disease. This is because chronic conditions can lead to changes in metabolism, diet, and physical activity, which can contribute to the formation of gallstones. Additionally, certain medications used to treat chronic conditions may also increase the risk of gallstones. Therefore, there is a plausible mechanism 

Processing rows:   2%|█▊                                                                           | 2/84 [06:20<4:26:54, 195.30s/it]

Failed parse
 {"answer": "A", "explanation": "Alanine aminotransferase (ALT) is an enzyme found in the liver and is often used as a marker for liver damage or inflammation. In the context of gallbladder disease, ALT levels can be elevated due to biliary obstruction, which can occur in conditions such as cholelithiasis, cholecystitis, and cholangitis. The elevation of ALT levels in these conditions can be attributed to the backup of bile and increased pressure in the biliary system, leading to liver cell damage and subsequent release of ALT into the bloodstream. Therefore, there is a plausible mechanism for ALT to be a potential predictive feature or risk factor for gallbladder disease."} {"answer": "A", "explanation": "Alanine aminotransferase (ALT) is an enzyme found in the liver and is often used as a marker for liver damage or inflammation. In the context of gallbladder disease, ALT levels can be elevated due to biliary obstruction, which can occur in conditions such as cholelithias

...

The feature 'Cholesterol in Chylomicrons and Extremely Large VLDL' makes sense as a potential predictive feature for the target diseases: Cholelithiasis, Gallstones, Gallbladder disease, Cholecystitis, and Cholangitis. The plausible mechanism for this feature to affect the target is as follows:

1. Cholesterol in chylomicrons and extremely large VLDL particles can contribute to the formation of cholesterol stones, which are a common type of gallstone. This is supported by the information in Document [0], which states that cholesterol stones make up approximately 75% of the secondary common bile duct stones in the United States.

2. The presence of cholesterol in chylomicrons and extremely large VLDL particles can lead to increased bile cholesterol due to hepatic secretion and excess triglycerides. This can promote the secretion of gallbladder mucin, contribute to a larger fasting gallbladder volume, reduce postprandial emptying, and impair intestinal motility, as mentioned in Document 

Processing rows:  99%|███████████████████████████████████████████████████████████████████████████ | 83/84 [6:02:11<08:24, 504.60s/it]

The feature 'Job code at visit clerks' with a p-value of 0.08 and a correlation of -0.003 does not make sense as a potential predictive feature or risk/protective factor for the target disease: Cholelithiasis, Gallstones, Gallbladder disease, cholecystitis, or Cholangitis. There is no plausible mechanism or link between a person's job code and the development of these diseases. The p-value is not statistically significant (usually a p-value of less than 0.05 is considered significant), and the correlation is very weak, indicating no strong relationship between the feature and the target disease.


Processing rows: 100%|████████████████████████████████████████████████████████████████████████████| 84/84 [6:05:38<00:00, 261.17s/it]

Failed parse
 {"answer": "B", "explanation": "The feature 'Other diseases of respiratory system, NEC' does not have a plausible link, explanation, or mechanism to affect the target disease 'Cholelithiasis OR Gallstones OR Gallbladder disease OR cholecystitis OR Cholangitis'. The respiratory system and the biliary system are two distinct systems in the body, and diseases affecting one system do not typically have a direct impact on the other. The p-value of 0.303 and correlation of -0.002 also suggest that there is no significant relationship between the feature and the target disease."} {"answer": "B", "explanation": "The feature 'Other diseases of respiratory system, NEC' does not have a plausible link, explanation, or mechanism to affect the target disease 'Cholelithiasis OR Gallstones OR Gallbladder disease OR cholecystitis OR Cholangitis'. The respiratory system and the biliary system are two distinct systems in the body, and diseases affecting one system do not typically have a di

In [12]:
answer

' {"answer": "B", "explanation": "The feature \'Other diseases of respiratory system, NEC\' does not have a plausible link, explanation, or mechanism to affect the target disease \'Cholelithiasis OR Gallstones OR Gallbladder disease OR cholecystitis OR Cholangitis\'. The respiratory system and the biliary system are two distinct systems in the body, and diseases affecting one system do not typically have a direct impact on the other. The p-value of 0.303 and correlation of -0.002 also suggest that there is no significant relationship between the feature and the target disease."} {"answer": "B", "explanation": "The feature \'Other diseases of respiratory system, NEC\' does not have a plausible link, explanation, or mechanism to affect the target disease \'Cholelithiasis OR Gallstones OR Gallbladder disease OR cholecystitis OR Cholangitis\'. The respiratory system and the biliary system are two distinct systems in the body, and diseases affecting one system do not typically have a direct

In [13]:
# json.loads(re.search('{.+}', re.sub(r'{ ', '{', answer), re.IGNORECASE).group(0))["answer"]


In [17]:
df_plausible_pred_big = pd.DataFrame({"question":df2["question"],"Plausible/Makes Sense?":df2["Plausible/Makes Sense?"] ,
                                    "pred_novel":big_plausible_pred,"pred_plausible_cot":big_plausible_cot,"plausible_full":big_plausible_answers_full,
                                   })

df_plausible_pred_big["pred_plausible"] = df_plausible_pred_big["pred_novel"].str.replace("[^A-B]"," ",case=False,regex=True).str.strip()
df_plausible_pred_big["pred_plausible"] = df_plausible_pred_big["pred_plausible"].str.startswith("A") # NOTE! Fails to account for bad formats!
# df_plausible_pred_big["pred_plausible"] = df_plausible_pred_big["pred_novel"] == "A"

display(df_plausible_pred_big)
#ALT? 
# df_plausible_pred_big["pred_plausible"] = df_plausible_pred_big["pred_novel"].str.contains("B\.",regex=True)

print(classification_report(df_plausible_pred_big["Plausible/Makes Sense?"],df_plausible_pred_big["pred_plausible"]))
print(df_plausible_pred_big[["Plausible/Makes Sense?","pred_plausible"]].corr())
if not FAST: 
    print("rAUC",round(100*roc_auc_score(df_plausible_pred_big["Plausible/Makes Sense?"],df_plausible_pred_big["pred_plausible"]),3))
if SAVE_OUTPUT:
    # df_plausible_pred_big.to_csv(PLAUSIBLE_OUTPUT_FILENAME,index=False)
    df_plausible_pred_big.to_csv("plausible_res_big.csv",index=False)


,question,Plausible/Makes Sense?,pred_novel,pred_plausible_cot,plausible_full,pred_plausible
0,"Given the feature 'Long-standing illness, disa...",True,""": ""A"", ""explanation"": ""Long-standing illness,...",""": ""Long-standing illness, disability, or infi...","{""answer"": ""A"", ""explanation"": ""Long-standing...",True
1,Given the feature 'Alanine aminotransferase'; ...,True,""": ""A"", ""explanation"": ""Alanine aminotransfera...",""": ""Alanine aminotransferase (ALT) is an enzym...","{""answer"": ""A"", ""explanation"": ""Alanine amino...",True
2,Given the feature 'Mouth/teeth dental problems...,True,""": ""B"", ""explanation"": ""While there may be a w...",for this relationship. The p-value of 0.0 ind...,"{""answer"": ""B"", ""explanation"": ""While there m...",False
3,Given the feature 'Overall health rating Fair'...,False,""": ""A"", ""explanation"": ""The feature 'Overall h...",""": ""The feature 'Overall health rating Fair' c...","{""answer"": ""A"", ""explanation"": ""The feature '...",True
4,Given the feature 'Standard PRS for alzheimer'...,True,B,The feature 'Standard PRS for Alzheimer's dise...,"{""answer"": ""B"", ""explanation"": ""The feature '...",False
...,...,...,...,...,...,...
79,Given the feature '3-Hydroxybutyrate'; (p-valu...,False,""": ""B"", ""explanation"": ""The feature '3-Hydroxy...",""": ""The feature '3-Hydroxybutyrate' with a p-v...","{""answer"": ""B"", ""explanation"": ""The feature '...",False
80,Given the feature 'Hallux valgus (Bunion)'; (p...,False,B,Hallux valgus (bunion) is a deformity of the j...,"{""answer"": ""B"", ""explanation"": ""Hallux valgus...",False
81,Given the feature 'Cholesterol in Chylomicrons...,False,A,The feature 'Cholesterol in Chylomicrons and E...,"{""answer"": ""A"", ""explanation"": ""The feature '...",True
82,Given the feature 'Job code at visit clerks'; ...,False,B,The feature 'Job code at visit clerks' with a ...,"{""answer"": ""B"", ""explanation"": ""The feature '...",False


              precision    recall  f1-score   support

       False       0.61      0.82      0.70        45
        True       0.65      0.38      0.48        39

    accuracy                           0.62        84
   macro avg       0.63      0.60      0.59        84
weighted avg       0.63      0.62      0.60        84

                        Plausible/Makes Sense?  pred_plausible
Plausible/Makes Sense?                1.000000        0.231334
pred_plausible                        0.231334        1.000000
rAUC 60.342


In [ ]:
# df_join = pd.concat([df_novel_pred_small[["Interesting?","pred_Interesting"]],df_novel_pred["pred_Interesting"].rename("pred2")],axis=1)
# df_join = df_join.astype(int)
# df_join["pred"] = df_join[["pred_Interesting","pred2"]].mean(axis=1)

# print(classification_report(df_join["Interesting?"],df_join["pred"]>0))
# print(round(100*roc_auc_score(df_join["Interesting?"],df_join["pred"]),4))
